Instalamos las dependencias

In [ ]:
!pip install requests
!pip install BeautifulSoup4
!pip install chromadb sentence-transformers langchain_huggingface langchain_chroma

In [ ]:
!pip install langchain-ollama

In [ ]:
!pip install langchain

In [22]:
import requests
from bs4 import BeautifulSoup
from langchain_ollama.chat_models import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [17]:
import requests
from bs4 import BeautifulSoup

# URL de la página de Messi en Wikipedia en inglés
url = 'https://en.wikipedia.org/wiki/Lionel_Messi'

# Petición a la API
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    page_text = soup.get_text(separator=' ', strip=True)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
    web_texts = text_splitter.split_text(page_text)
    print(web_texts)
else:
    print(f'No se pudo accede a la página: {response.status_code}')

['Lionel Messi - Wikipedia Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk', "Contributions Talk Contents move to sidebar hide (Top) 1 Early life 2 Youth career Toggle Youth career subsection 2.1 Newell's Old Boys (1995–2000) 2.2 Barcelona (2000–2004) 3 Club career Toggle Club career subsection 3.1 Barcelona (2004–2021) 3.1.1 2004–2008: Rise to the first team 3.1.2 2008–2012: Success under Pep Guardiola 3.1.3 2012–2014: Record-breaking year and Messidependencia 3.1.4", '3.1.4 2014–2017: Arrival of Luis Enrique and birth of MSN 3.1.5 2017–2021: Final years at Barcelona 3.2 Paris Saint-Germain (2021–2023) 3.3 Inter Miami (2023–present) 4 International career 5 Player

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/bda/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
bbdd_vector = Chroma.from_texts(
    texts=web_texts,
    collection_name="players",
    embedding=embeddings,
)

In [24]:
retriever = bbdd_vector.as_retriever()

In [30]:
question = "¿Who is Lionel Messi?"
docs = bbdd_vector.similarity_search(question, k=5)

NameError: name 'bbdd_vector' is not defined

In [ ]:
# Prompt: Se define una plantilla de pregunta-respuesta que toma un contexto y una pregunta como entrada
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = "llama3.2"
modelo = ChatOllama(model=llm)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}| prompt| modelo| StrOutputParser()
)
chain.invoke("Who is Lionel Messi?")